# 첫 번째 도구 강제 호출

이 노트북에서는 **첫 번째 도구를 강제로 호출**하는 에이전트를 만듭니다.

## 왜 첫 번째 도구를 강제할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구 강제 호출의 필요성                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 에이전트:                                                   │
│   "한국의 수도는 어디인가요?" → LLM이 바로 답변 (도구 안 씀)       │
│   (LLM이 이미 알고 있으면 도구를 사용하지 않음)                    │
│                                                                    │
│   문제:                                                            │
│   • 최신 정보가 필요한 질문인데 LLM의 오래된 지식으로 답변         │
│   • 항상 검색을 해야 하는 시스템인데 LLM이 검색 안 함              │
│                                                                    │
│   해결:                                                            │
│   첫 번째 노드에서 무조건 검색 도구를 호출하도록 강제!             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처 비교

```
┌────────────────────────────────────────────────────────────────────┐
│                    일반 vs 강제 호출 아키텍처                       │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [일반 에이전트]              [강제 호출 에이전트]                │
│                                                                    │
│   START                        START                               │
│     │                            │                                 │
│     ▼                            ▼                                 │
│   model ─────┐               first_model ← 강제로 검색 호출        │
│     │        │                   │                                 │
│   (조건)    END                  ▼                                 │
│     │                         tools                                │
│     ▼                            │                                 │
│   tools                          ▼                                 │
│     │                         model ─────┐                         │
│     └────────┘                   │       │                         │
│                                (조건)   END                        │
│                                  │                                 │
│                                  ▼                                 │
│                               tools                                │
│                                  │                                 │
│                                  └───────┘                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 도구 정의

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

print("✅ 도구 정의 완료")

# 3. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)

print("✅ LLM 설정 완료")

# 4. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")

# 5. 노드 정의 (핵심: first_model)

```
┌────────────────────────────────────────────────────────────────────┐
│                    ToolCall 직접 생성하기                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적으로 LLM이 tool_calls를 생성하지만,                        │
│   우리가 직접 ToolCall을 만들어서 강제로 도구 호출 가능!           │
│                                                                    │
│   ToolCall 구성요소:                                               │
│   • name: 호출할 도구 이름                                         │
│   • args: 도구에 전달할 인자 (딕셔너리)                            │
│   • id: 고유 식별자 (uuid 사용)                                    │
│                                                                    │
│   예시:                                                            │
│   ToolCall(                                                        │
│       name='duckduckgo_search',                                    │
│       args={'query': '사용자 질문'},                               │
│       id='abc123'                                                  │
│   )                                                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from uuid import uuid4
from langchain_core.messages import AIMessage, HumanMessage, ToolCall

def model_node(state: State) -> State:
    """
    일반 LLM 호출 노드 (두 번째 이후 사용)
    """
    res = model.invoke(state['messages'])
    return {'messages': res}


def first_model(state: State) -> State:
    """
    첫 번째 노드: 검색 도구 강제 호출
    
    1. 사용자 질문을 가져옴
    2. ToolCall을 직접 생성 (검색 도구)
    3. AIMessage에 tool_calls 포함하여 반환
    """
    # 사용자 질문 가져오기
    query = state['messages'][-1].content
    
    # 검색 도구 호출을 직접 생성
    search_tool_call = ToolCall(
        name='duckduckgo_search',  # 도구 이름
        args={'query': query},      # 도구 인자
        id=uuid4().hex              # 고유 ID
    )
    
    # tool_calls가 포함된 AIMessage 반환
    return {
        'messages': AIMessage(
            content='',  # 내용은 비워둠
            tool_calls=[search_tool_call]  # 도구 호출 정보
        )
    }

print("✅ 노드 정의 완료")
print("   - first_model: 첫 번째로 검색 강제 호출")
print("   - model_node: 이후 LLM 판단에 따라 도구 사용")

# 6. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

# 노드 추가 (first_model 추가됨)
builder.add_node('first_model', first_model)  # 첫 번째 노드
builder.add_node('model', model_node)         # 일반 모델 노드
builder.add_node('tools', ToolNode(tools))

# 엣지 추가
builder.add_edge(START, 'first_model')         # 시작 → first_model
builder.add_edge('first_model', 'tools')       # first_model → tools (항상)
builder.add_conditional_edges('model', tools_condition)  # model은 조건부
builder.add_edge('tools', 'model')             # tools → model

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. 에이전트 실행

In [ ]:
# 테스트 실행
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 강제 검색 에이전트 실행 ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if node_name == 'first_model':
            print("  → 검색 도구 강제 호출!")
        print()

In [ ]:
# 최종 결과
result = graph.invoke(input_data)

print("=== 최종 응답 ===")
print(result['messages'][-1].content)

# 8. 일반 에이전트와 비교

LLM이 이미 알 법한 질문으로 테스트해봅니다.

In [ ]:
# 일반 에이전트 만들기 (비교용)
normal_builder = StateGraph(State)
normal_builder.add_node('model', model_node)
normal_builder.add_node('tools', ToolNode(tools))
normal_builder.add_edge(START, 'model')
normal_builder.add_conditional_edges('model', tools_condition)
normal_builder.add_edge('tools', 'model')
normal_graph = normal_builder.compile()

print("✅ 일반 에이전트 (비교용) 생성 완료")

In [ ]:
# 비교 테스트
test_question = "한국의 수도는 어디인가요?"

print("=" * 60)
print(f"질문: {test_question}")
print("=" * 60)

# 일반 에이전트
print("\n[일반 에이전트]")
normal_result = normal_graph.invoke({'messages': [HumanMessage(test_question)]})
normal_tools_used = [tc['name'] for msg in normal_result['messages'] 
                     if hasattr(msg, 'tool_calls') and msg.tool_calls 
                     for tc in msg.tool_calls]
print(f"  사용된 도구: {normal_tools_used if normal_tools_used else '없음'}")

# 강제 검색 에이전트
print("\n[강제 검색 에이전트]")
forced_result = graph.invoke({'messages': [HumanMessage(test_question)]})
forced_tools_used = [tc['name'] for msg in forced_result['messages'] 
                     if hasattr(msg, 'tool_calls') and msg.tool_calls 
                     for tc in msg.tool_calls]
print(f"  사용된 도구: {forced_tools_used if forced_tools_used else '없음'}")

print("\n→ 강제 검색 에이전트는 항상 검색부터 시작!")

---

## 정리: 첫 번째 도구 강제 호출

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    ToolCall 직접 생성                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   from langchain_core.messages import ToolCall                      │
│   from uuid import uuid4                                            │
│                                                                     │
│   tool_call = ToolCall(                                             │
│       name='도구_이름',          # 호출할 도구                      │
│       args={'key': 'value'},     # 전달할 인자                      │
│       id=uuid4().hex             # 고유 ID                          │
│   )                                                                 │
│                                                                     │
│   # AIMessage에 포함하여 반환                                       │
│   AIMessage(content='', tool_calls=[tool_call])                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
def first_model(state: State) -> State:
    query = state['messages'][-1].content
    
    # 도구 호출 직접 생성
    search_tool_call = ToolCall(
        name='duckduckgo_search',
        args={'query': query},
        id=uuid4().hex
    )
    
    return {
        'messages': AIMessage(content='', tool_calls=[search_tool_call])
    }

# 그래프에서 first_model → tools 직접 연결 (조건 없이)
builder.add_edge('first_model', 'tools')
```

### 사용 시나리오

| 시나리오 | 설명 |
|----------|------|
| 뉴스 검색 봇 | 항상 최신 뉴스 검색 후 답변 |
| 실시간 정보 | 날씨, 주가 등 항상 검색 필요 |
| RAG 시스템 | 항상 문서 검색 후 답변 |
| 팩트체크 | 답변 전 사실 확인 필수 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**많은 도구 중에서 관련 도구만 선택**하는 방법을 배웁니다. (05-06번 노트북)